In [ ]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jlhammel/geog-510/blob/four/book/labs/lab_04.ipynb)

# Lab 5

## Exercise 1: Calculating Distances with Functions

- Define a function `calculate_distance` that takes two geographic coordinates (latitude and longitude) and returns the distance between them using the Haversine formula.
- Use this function to calculate the distance between multiple pairs of coordinates.

In [287]:
from math import radians, sin, cos, sqrt, atan2

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = (
        sin(dlat / 2) ** 2
        + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    )
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    calculate_distance = R * c
    return calculate_distance


calculate_distance = haversine(68.6385, 122.6937, 87.0522, -18.2333)
print(f"Distance: {calculate_distance:.2f} km")

## Exercise 2: Batch Distance Calculation

- Create a function `batch_distance_calculation` that accepts a list of coordinate pairs and returns a list of distances between consecutive pairs.
- Test the function with a list of coordinates representing several cities.

In [ ]:
def batch_haversine(coord_list):
    batch_distance_calculation = []
    for i in range(len(coord_list) - 1):
        lat1, lon1 = coord_list[i]
        lat2, lon2 = coord_list[i + 1]
        distance = haversine(lat1, lon1, lat2, lon2)
        batch_distance_calculation.append(distance)
    return batch_distance_calculation


# Example usage
coordinates = [(35.9653, 83.9233), (35.6764, 139.6500), (9.0820, 8.6753)]
batch_distance_calculation = batch_haversine(coordinates)
print(f"Distances: {batch_distance_calculation}")

## Exercise 3: Creating and Using a Point Class

- Define a `Point` class to represent a geographic point with attributes `latitude`, `longitude`, and `name`.
- Add a method `distance_to` that calculates the distance from one point to another.
- Instantiate several `Point` objects and calculate the distance between them.

In [ ]:
class Point:
    def __init__(self, latitude, longitude, name=None):
        self.latitude = latitude
        self.longitude = longitude
        self.name = name

    def distance_to(self, other_point):
        return haversine(
            self.latitude, self.longitude, other_point.latitude, other_point.longitude
        )



point1 = Point(35.9606, 83.9207, "Knoxville")
point2 = Point(51.5072, 0.1276, "London")
print(
    f"Distance from {point1.name} to {point2.name}: {point1.distance_to(point2):.2f} km"
)

## Exercise 4: Reading and Writing Files

- Write a function `read_coordinates` that reads a file containing a list of coordinates (latitude, longitude) and returns them as a list of tuples.
- Write another function `write_coordinates` that takes a list of coordinates and writes them to a new file.
- Ensure that both functions handle exceptions, such as missing files or improperly formatted data.

In [ ]:
s_data = """[35.6895,139.6917]
[64.052,-11.2437]
[62.5074,-10.1278]
[-37.8688,144.2093]
[58.8566,12.3522]"""

output_file = "coord.txt"

try:
    with open(output_file, "w") as file:
        file.write(s_data)
    print(f"File '{output_file}' has been created successfully.")
except Exception as e:
    print(f"An error occurred while creating the file: {e}")

In [ ]:
read_coordinates = "coord.txt"
write_coordinates = "output_coord.txt"

try:
    with open(read_coordinates, "r") as infile:
        coordinates = infile.readlines()

    with open(write_coordinates, "w") as outfile:
        for line in coordinates:
            lat, lon = line.strip().split(",")
            outfile.write(f"Latitude: {lat}, Longitude: {lon}\n")

    print(f"Coordinates have been written to {write_coordinates}")
except FileNotFoundError:
    print(f"Error: The file {read_coordinates} was not found.")

## Exercise 5: Processing Coordinates from a File

- Create a function that reads coordinates from a file and uses the `Point` class to create `Point` objects.
- Calculate the distance between each consecutive pair of points and write the results to a new file.
- Ensure the function handles file-related exceptions and gracefully handles improperly formatted lines.

In [349]:
import os
from shapely.geometry import Point
from pyproj import Geod

input_file = "output_coords.txt"
output_file = "coordinates_out.txt"

def process_coordinates(input_file, output_file):
    points = []
    geod = Geod(ellps="WGS84")  
    
    try:
        with open(input_file, 'r') as file:
            for line_num, line in enumerate(file, 1):
                try:
                    parts = line.strip().split(',')
                    if len(parts) != 2:
                        raise ValueError("Line doesn't have exactly two values.")
                    lon, lat = map(float, parts)
                    points.append(Point(lon, lat))
                except ValueError as ve:
                    print(f"Skipping line {line_num}: {ve}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' does not exist.")
        return
    except IOError as e:
        print(f"I/O error occurred: {e}")
        return

    if len(points) < 2:
        print("Not enough valid points to calculate distances.")
        return


    distances = []
    for i in range(len(points) - 1):
        lon1, lat1 = points[i].x, points[i].y
        lon2, lat2 = points[i + 1].x, points[i + 1].y
        _, _, distance = geod.inv(lon1, lat1, lon2, lat2)
        distances.append((i, i + 1, distance)) 

    try:
        with open(output_file, 'w') as file:
            file.write("PointA_Index,PointB_Index,Distance_meters\n")
            for idx_a, idx_b, dist in distances:
                file.write(f"{idx_a},{idx_b},{dist:.2f}\n")
        print(f"Distance results written to '{output_file}'.")
    except IOError as e:
        print(f"Failed to write to '{output_file}': {e}")



## Exercise 6: Exception Handling in Data Processing

- Modify the `batch_distance_calculation` function to handle exceptions that might occur during the calculation, such as invalid coordinates.
- Ensure the function skips invalid data and continues processing the remaining data.

In [296]:
def batch_distance_calculation(line):
    try:
        lat, lon = line.strip().split(",")
        lat = float(lat) 
        lon = float(lon)
        return lat, lon
    except ValueError as e:
        print(f"Error: {e}. Could not parse line: {line.strip()}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

## Exercise 7: NumPy Array Operations and Geospatial Coordinates

In this exercise, you will work with NumPy arrays representing geospatial coordinates (latitude and longitude) and perform basic array operations.

1. Create a 2D NumPy array containing the latitude and longitude of the following cities: Tokyo (35.6895, 139.6917), New York (40.7128, -74.0060), London (51.5074, -0.1278), and Paris (48.8566, 2.3522).
2. Convert the latitude and longitude values from degrees to radians using np.radians().
3. Calculate the element-wise difference between Tokyo and the other cities' latitude and longitude in radians.

In [297]:
import numpy as np

In [298]:
class Point:
    def __init__(self, latitude, longitude, name=None):
        self.latitude = latitude
        self.longitude = longitude
        self.name = name

    def distance_to(self, other_point):
        return haversine(
            self.latitude, self.longitude, other_point.latitude, other_point.longitude
        )

In [ ]:
# Creating a 2D array
arr_2d = np.array([["Tokyo", 35.6895, 139.6917, "New York", 40.7128, -74.0060 ], ["London", 51.5074, -0.1278, "Paris",48.8566, 2.3522 ]])
print(f"2D Array:\n{arr_2d}")

In [ ]:
coords = np.array([[35.6895, 139.6917],[40.7128, -74.0060],[51.5074, -0.1278],[48.8566, 2.3522]])
coords_rad = np.radians(coords)
print(f"Coordinates in radians:\n{coords_rad}")

In [ ]:
point1 = Point(6.22899283e-01,  2.43808010e+00, "Tokyo")
point2 = Point(8.98973719e-01, -2.23053078e-03, "London")
point3 = Point(7.10572408e-01, -1.29164837e+00, "New York")
point4 = Point(8.52708531e-01,  4.10536347e-02, "Paris")
print(
    f"Distance from {point1.name} to {point2.name}: {point1.distance_to(point2):.2f} km"
)
print(
    f"Distance from {point1.name} to {point3.name}: {point1.distance_to(point3):.2f} km"
    
)
print(
    f"Distance from {point1.name} to {point4.name}: {point1.distance_to(point4):.2f} km"
)

## Exercise 8: Pandas DataFrame Operations with Geospatial Data

In this exercise, you'll use Pandas to load and manipulate a dataset containing city population data, and then calculate and visualize statistics.

1. Load the world cities dataset from this URL using Pandas: https://github.com/opengeos/datasets/releases/download/world/world_cities.csv
2. Display the first 5 rows and check for missing values.
3. Filter the dataset to only include cities with a population greater than 1 million.
4. Group the cities by their country and calculate the total population for each country.
5. Sort the cities by population in descending order and display the top 10 cities.

In [302]:
import pandas as pd

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/world/world_cities.csv"
df = pd.read_csv(url)
df.head(5)

In [ ]:
df_filtered = df[df["population"] > 1000000]
print(df_filtered)

In [ ]:
df_grouped = df.groupby("country")["population"].sum()
print(f"Total Population by Country:\n{df_grouped}")

In [ ]:
df_sorted = df.sort_values(by='population', ascending=False)
df_sorted.head(10)

## Exercise 9: Creating and Manipulating GeoDataFrames with GeoPandas

This exercise focuses on creating and manipulating GeoDataFrames, performing spatial operations, and visualizing the data.

1. Load the New York City building dataset from the GeoJSON file using GeoPandas: https://github.com/opengeos/datasets/releases/download/places/nyc_buildings.geojson
2. Create a plot of the building footprints and color them based on the building height (use the `height_MS` column).
3. Create an interactive map of the building footprints and color them based on the building height (use the `height_MS` column).
4. Calculate the average building height (use the `height_MS` column).
5. Select buildings with a height greater than the average height.
6. Save the GeoDataFrame to a new GeoJSON file.

In [307]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/places/nyc_buildings.geojson"
gdf = gpd.read_file(url)
gdf.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(column="height_MS", cmap="viridis", legend=True, ax=ax)
ax.set_title("NYC Buildings Colored by Height")
plt.show()

In [ ]:
gdf.explore(column = "height_MS", categorical = True)

In [ ]:
average_height = gdf["height_MS"].mean()
print(f"Average Building Height: {average_height:.2f} meters")

In [ ]:
avght = gdf[gdf["height_MS"] > average_height]
print(avght)

In [313]:
gdf.to_file(output_file, driver = "GeoJSON")

## Exercise 10: Combining NumPy, Pandas, and GeoPandas

This exercise requires you to combine the power of NumPy, Pandas, and GeoPandas to analyze and visualize spatial data.

1. Use Pandas to load the world cities dataset from this URL: https://github.com/opengeos/datasets/releases/download/world/world_cities.csv
2. Filter the dataset to include only cities with latitude values between -40 and 60 (i.e., cities located in the Northern Hemisphere or near the equator).
3. Create a GeoDataFrame from the filtered dataset by converting the latitude and longitude into geometries.
4. Reproject the GeoDataFrame to the Mercator projection (EPSG:3857).
5. Calculate the distance (in meters) between each city and the city of Paris.
6. Plot the cities on a world map, coloring the points by their distance from Paris.

In [339]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [331]:
url = "https://github.com/opengeos/datasets/releases/download/world/world_cities.csv"
df = pd.read_csv(url)
df.head()

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.5833,32.5333,75000
1,2,Fort Portal,UGA,0.6710,30.2750,42670
2,3,Potenza,ITA,40.6420,15.7990,69060
3,4,Campobasso,ITA,41.5630,14.6560,50762
4,5,Aosta,ITA,45.7370,7.3150,34062


In [342]:
lower_bound = -40
upper_bound = 60
filtered_df_between = df[df['latitude'].between(lower_bound, upper_bound)]
gdf = gpd.GeoDataFrame(filtered_df_between, geometry=gpd.points_from_xy(filtered_df_between.longitude, filtered_df_between.latitude), crs="EPSG:3857")
print("Filtered with .between():")
print(filtered_df_between)

Filtered with .between():
        id            name country  latitude  longitude  population
0        1           Bombo     UGA   0.58330   32.53330       75000
1        2     Fort Portal     UGA   0.67100   30.27500       42670
2        3         Potenza     ITA  40.64200   15.79900       69060
3        4      Campobasso     ITA  41.56300   14.65600       50762
4        5           Aosta     ITA  45.73700    7.31500       34062
...    ...             ...     ...       ...        ...         ...
1244  1245  Rio de Janeiro     BRA -22.92502  -43.22502    11748000
1245  1246       Sao Paulo     BRA -23.55868  -46.62502    18845000
1246  1247          Sydney     AUS -33.92001  151.18518     4630000
1247  1248       Singapore     SGP   1.29303  103.85582     5183700
1248  1249       Hong Kong     CHN  22.30498  114.18501     7206000

[1132 rows x 6 columns]


In [346]:
gdf = [Point(x, y) for x,y in zip(filtered_df_between["longitude"], filtered_df_between["latitude"])]
gdf = gpd.GeoDataFrame(filtered_df_between, geometry=gpd.points_from_xy(filtered_df_between.longitude, filtered_df_between.latitude), crs="EPSG:3857")
gdf.head()

,id,name,country,latitude,longitude,population,geometry
0,1,Bombo,UGA,0.5833,32.5333,75000,POINT (32.533 0.583)
1,2,Fort Portal,UGA,0.6710,30.2750,42670,POINT (30.275 0.671)
2,3,Potenza,ITA,40.6420,15.7990,69060,POINT (15.799 40.642)
3,4,Campobasso,ITA,41.5630,14.6560,50762,POINT (14.656 41.563)
4,5,Aosta,ITA,45.7370,7.3150,34062,POINT (7.315 45.737)


In [347]:
paris_geometry = gdf.loc[gdf["name"] == "Paris", "geometry"].values[0]
gdf["distance_to_paris"] = gdf.distance(paris_geometry)
gdf["distance_to_paris"] = gdf["distance_to_paris"]
print(gdf)

        id            name country  latitude  longitude  population  \
0        1           Bombo     UGA   0.58330   32.53330       75000   
1        2     Fort Portal     UGA   0.67100   30.27500       42670   
2        3         Potenza     ITA  40.64200   15.79900       69060   
3        4      Campobasso     ITA  41.56300   14.65600       50762   
4        5           Aosta     ITA  45.73700    7.31500       34062   
...    ...             ...     ...       ...        ...         ...   
1244  1245  Rio de Janeiro     BRA -22.92502  -43.22502    11748000   
1245  1246       Sao Paulo     BRA -23.55868  -46.62502    18845000   
1246  1247          Sydney     AUS -33.92001  151.18518     4630000   
1247  1248       Singapore     SGP   1.29303  103.85582     5183700   
1248  1249       Hong Kong     CHN  22.30498  114.18501     7206000   

                     geometry  distance_to_paris  
0        POINT (32.533 0.583)          56.950183  
1        POINT (30.275 0.671)          55.709

In [348]:
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt




gdf.explore(column = "distance_to_paris", basemap= 'OpenStreetMap', categorical=True)


## Submission Requirements

Complete the exercises above and and upload the notebook to your GitHub repository. Make sure the notebook has a Colab badge at the top so that it can be easily opened in Google Colab. Submit the URL of the notebook to Canvas.